In [26]:
import pandas as pd
import numpy as np

data = pd.read_csv('../input/nypd-summons-historic/NYPD_Criminal_Court_Summons__Historic_.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [29]:
data[data['LAW_SECTION_NUMBER'] == '9999'].OFFENSE_DESCRIPTION.value_counts()

OTHER ADMIN CODE                187286
OTHER NYS TRANSPORTATION         51650
OTHER VTL                        43437
OTHER PARK REGS                  27645
OTHER ABC                        25959
                                 ...  
FAIL TO DIPLAY OCCUPANCY             1
panhandling terminal                 1
DISCHARGE W/OUT PERMIT               1
UNLIC LIVERY                         1
LOITERING ON  SCHOOL GROUNDS         1
Name: OFFENSE_DESCRIPTION, Length: 3072, dtype: int64

In [3]:
data.head()

,SUMMONS_KEY,SUMMONS_DATE,OFFENSE_DESCRIPTION,LAW_SECTION_NUMBER,LAW_DESCRIPTION,SUMMONS_CATEGORY_TYPE,AGE_GROUP,SEX,RACE,JURISDICTION_CODE,BORO,PRECINCT_OF_OCCUR,X_COORDINATE_CD,Y_COORDINATE_CD,Latitude,Longitude,Lon_Lat
0,192724412,01/24/2019,UNINSURED VEHICLE,319,VTL,VTL,UNKNOWN,NaN,NaN,0,NEW YORK,18,990060.0,217404.0,40.763407,-73.979028,POINT (-73.97902801199996 40.76340651400006)
1,193633599,02/15/2019,OTHER VTL,9999,VTL,VTL,UNKNOWN,NaN,NaN,0,BROOKLYN,78,991048.0,187513.0,40.681362,-73.975492,POINT (-73.97549163099997 40.68136223600004)
2,194790153,03/17/2019,MAKES UNREASONABLE NOISE,240.20(2),NaN,NaN,25-44,M,WHITE HISPANIC,0,BROOKLYN,72,984058.0,178998.0,40.657993,-74.000693,POINT (-74.00069344599996 40.657993066000074)
3,198874018,06/24/2019,OTHER SANITARY CODE,9999,ABC,OTHER ABC,UNKNOWN,NaN,NaN,0,QUEENS,109,1031014.0,217293.0,40.762980,-73.831190,POINT (-73.83118971499994 40.76298008300006)
4,191809027,01/02/2019,"MARIJUANA, POSSESSION OF",221.05,Penal Law,MARIJUANA,25-44,M,BLACK,0,BRONX,44,1003989.0,244111.0,40.836690,-73.928667,POINT (-73.92866741799997 40.836689797000076)


In [4]:
data = data[['SUMMONS_KEY','SUMMONS_DATE','LAW_SECTION_NUMBER']]

In [5]:
data['SUMMONS_DATE'] = pd.to_datetime(data['SUMMONS_DATE'])
data['SUMMONS_YEAR'] = pd.DatetimeIndex(data['SUMMONS_DATE']).year
data.head()

,SUMMONS_KEY,SUMMONS_DATE,LAW_SECTION_NUMBER,SUMMONS_YEAR
0,192724412,2019-01-24,319,2019
1,193633599,2019-02-15,9999,2019
2,194790153,2019-03-17,240.20(2),2019
3,198874018,2019-06-24,9999,2019
4,191809027,2019-01-02,221.05,2019


In [6]:
lsn = data['LAW_SECTION_NUMBER'].value_counts().head(20).index.tolist()
lsn

['10-125',
 '9999',
 '1-03',
 '19-176',
 '240.205A',
 '221.05',
 '140.05',
 'CFR 49',
 '153.09',
 '240.20',
 '16-1186',
 '1212',
 '240.20-1',
 '240.205B',
 '240.20-2',
 '240.20-6',
 '16-118',
 '20-453',
 '24-218',
 '19-506']

In [7]:
data_top_lsn = data.loc[data['LAW_SECTION_NUMBER'].isin(lsn)]
data_top_lsn.shape

(4121427, 4)

In [8]:
sy = data_top_lsn['SUMMONS_YEAR'].value_counts().index.tolist()

In [9]:
type(sy)

list

In [10]:
sy.sort()
sy

[2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020]

In [11]:
data_pivot = pd.DataFrame()
for code in lsn:
    f_points = []
    for year in sy:
        val = data_top_lsn.loc[(data_top_lsn['SUMMONS_YEAR'] == year) &
                     (data_top_lsn['LAW_SECTION_NUMBER'] == code)].shape[0]
        f_points.append(val)
    data_pivot[code] = pd.Series(f_points)

    
data_pivot.to_csv('data_pivot_for_bcr.csv')

In [12]:
!pip install bar-chart-race

     |████████████████████████████████| 156 kB 592 kB/s            


In [13]:
data_pivot['year'] = pd.Series(sy)
data_pivot.head()

,10-125,9999,1-03,19-176,240.205A,221.05,140.05,CFR 49,153.09,240.20,...,1212,240.20-1,240.205B,240.20-2,240.20-6,16-118,20-453,24-218,19-506,year
0,95740,12651,0,21140,17643,9173,16961,0,18571,2305,...,8726,6412,5154,5326,4659,4491,9023,912,4616,2006
1,129165,55698,15879,23181,32900,9350,18644,2643,18965,27360,...,13976,8902,7950,8328,7412,6213,8264,0,5165,2007
2,137283,45347,21860,24552,27618,8910,17327,16261,18099,22808,...,12788,9710,7876,7958,7433,6310,6248,1,4467,2008
3,142308,42545,26606,24884,28846,8776,16253,18744,18295,22685,...,14856,11020,9731,9031,8960,7388,7618,5254,4958,2009
4,153372,41065,28328,27957,25455,8422,16781,8740,18252,17375,...,14009,10737,11074,8758,8511,6375,7820,9235,5464,2010


In [14]:
data_pivot.set_index('year', inplace=True)
data_pivot.head()

,10-125,9999,1-03,19-176,240.205A,221.05,140.05,CFR 49,153.09,240.20,16-1186,1212,240.20-1,240.205B,240.20-2,240.20-6,16-118,20-453,24-218,19-506
year,,,,,,,,,,,,,,,,,,,,
2006,95740,12651,0,21140,17643,9173,16961,0,18571,2305,139,8726,6412,5154,5326,4659,4491,9023,912,4616
2007,129165,55698,15879,23181,32900,9350,18644,2643,18965,27360,5546,13976,8902,7950,8328,7412,6213,8264,0,5165
2008,137283,45347,21860,24552,27618,8910,17327,16261,18099,22808,9259,12788,9710,7876,7958,7433,6310,6248,1,4467
2009,142308,42545,26606,24884,28846,8776,16253,18744,18295,22685,9631,14856,11020,9731,9031,8960,7388,7618,5254,4958
2010,153372,41065,28328,27957,25455,8422,16781,8740,18252,17375,10365,14009,10737,11074,8758,8511,6375,7820,9235,5464


In [22]:
dp2 = data_pivot.copy()
dp2.index = dp2.index.astype('str')
dp2.head()

,10-125,9999,1-03,19-176,240.205A,221.05,140.05,CFR 49,153.09,240.20,16-1186,1212,240.20-1,240.205B,240.20-2,240.20-6,16-118,20-453,24-218,19-506
year,,,,,,,,,,,,,,,,,,,,
2006,95740,12651,0,21140,17643,9173,16961,0,18571,2305,139,8726,6412,5154,5326,4659,4491,9023,912,4616
2007,129165,55698,15879,23181,32900,9350,18644,2643,18965,27360,5546,13976,8902,7950,8328,7412,6213,8264,0,5165
2008,137283,45347,21860,24552,27618,8910,17327,16261,18099,22808,9259,12788,9710,7876,7958,7433,6310,6248,1,4467
2009,142308,42545,26606,24884,28846,8776,16253,18744,18295,22685,9631,14856,11020,9731,9031,8960,7388,7618,5254,4958
2010,153372,41065,28328,27957,25455,8422,16781,8740,18252,17375,10365,14009,10737,11074,8758,8511,6375,7820,9235,5464


In [24]:
dp2.index.astype('str')

Index(['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020'],
      dtype='object', name='year')

In [25]:
import bar_chart_race as bcr
bcr.bar_chart_race(df=dp2, steps_per_period=20, period_length=800, sort='desc',
    n_bars=10,dpi=144,title='NYS Penal Law Violations Over The Years')

/opt/conda/lib/python3.7/site-packages/bar_chart_race/_make_chart.py:286: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(self.df_values.columns)
/opt/conda/lib/python3.7/site-packages/bar_chart_race/_make_chart.py:287: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([max_val] * len(ax.get_xticks()))


In [30]:
import bar_chart_race as bcr
bcr.bar_chart_race(df=dp2,filename = 'test_run_year.mp4', figsize=(5, 3), steps_per_period=20, period_length=800, sort='desc',
    n_bars=10,dpi=288,title='NYS Penal Law Violations Over The Years')

/opt/conda/lib/python3.7/site-packages/bar_chart_race/_make_chart.py:286: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(self.df_values.columns)
/opt/conda/lib/python3.7/site-packages/bar_chart_race/_make_chart.py:287: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([max_val] * len(ax.get_xticks()))
